In [13]:
import numpy as np
import konlpy
from konlpy.tag import Twitter
import os
from os import listdir
from collections import Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
device

device(type='cuda', index=0)

# Hyperparameter

In [16]:
TEXT_DATA_DIR = './newsData'
NUM_CATEGORY = 8
BATCH_SIZE=10
EPOCHS=12
SPLIT_RATE=0.8
NUM_TEST_PER_DIRECTORY = 960

In [17]:
# unknown token 추가하여 초기화
word2idx = {'<unk>':0,'<number>':1}
idx2word = {0:'<unk>',1:'<number>'}
vocab=Counter()
vocab.update(['<unk>','<number>'])

# 데이터 전처리

In [18]:
def Extract_word(directory):
    print("%s 작업중"%directory)
    count = 1
    for filename in listdir(directory):
        # train 파일로만 단어장을 만든다.
        if count > NUM_TEST_PER_DIRECTORY:
            break
        path = directory+'/'+filename
        
        file = open(path,'r',encoding='utf-8')
        doc = file.read()
        file.close()
        
        twit = Twitter()
        doc = twit.pos(doc)
        tag=['Noun','Foreign','Alpha']  
        tagNum=['Number']
        exception=['\n\n','\t','‘','”',',','ㆍ','·','…','“','’','【','】','‧','◀','▶','\xa0','∼',"”…","ㆍ‘",
            '것','했',"있다","등","기자","또","며","그"]    
        for pair in doc:
            if pair[1] in 'Number':
                vocab.update(['<number>'])
            if pair[1] in tag and pair[0] not in exception:
                vocab.update([pair[0]])       
        count += 1

In [19]:
# vocab 만들기, 이미 정의된 vocab이 있는 경우 생략한다.
def Make_vocab():
    if os.path.exists("vocab.txt"):
        print("이미 정의된 vocab을 불러옵니다.")
        file=open('vocab.txt','r',encoding='utf-8')
        doc=file.read()
        doc = doc.split('\n')
        file.close()
        return doc
    else: 
        for i in range(NUM_CATEGORY):
            Extract_word(TEXT_DATA_DIR+'/'+str(i))
        #tokens=[k for k,c in vocab.items() if c >= MIN_COUNT]      
        # vocab 저장
        data = '\n'.join(vocab)
        file=open('vocab.txt','w',encoding='utf-8')
        file.write(data)
        file.close()
        print('단어 수 :',len(vocab))
        print('빈출 상위 50 단어 :')
        print(vocab.most_common(50))
        return vocab

In [20]:
vocab = Make_vocab()

./newsData/0 작업중


C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


./newsData/1 작업중
./newsData/2 작업중
./newsData/3 작업중
./newsData/4 작업중
./newsData/5 작업중
./newsData/6 작업중
./newsData/7 작업중
단어 수 : 53867
빈출 상위 50 단어 :
[('<number>', 135677), ('이', 15491), ('수', 15274), ('고', 14593), ('말', 10977), ('전', 8881), ('한국', 8259), ('미국', 6390), ('위', 6001), ('지난', 5680), ('대한', 5499), ('때', 5393), ('대해', 5096), ('대통령', 5055), ('서울', 5046), ('를', 5032), ('명', 4920), ('정부', 4909), ('중', 4736), ('의', 4695), ('더', 4586), ('때문', 4504), ('날', 4483), ('의원', 4394), ('위해', 4357), ('문제', 4332), ('중국', 4302), ('사람', 4237), ('시장', 3991), ('관련', 3979), ('이번', 3956), ('통해', 3918), ('로', 3812), ('우리', 3742), ('안', 3665), ('시간', 3651), ('개', 3623), ('대표', 3512), ('이후', 3504), ('경기', 3449), ('상황', 3406), ('김', 3402), ('생각', 3391), ('제', 3377), ('라며', 3367), ('내', 3329), ('경우', 3313), ('점', 3127), ('경제', 3001), ('정책', 2989)]


In [21]:
vocab

Counter({'<unk>': 1,
         '<number>': 135677,
         '동남아': 167,
         '담당': 495,
         '北': 94,
         '최희': 21,
         '철': 219,
         '부상': 606,
         '베이징': 243,
         '도착': 379,
         '싱가포르': 348,
         '행': 263,
         '주목': 777,
         '최': 1435,
         '행선': 22,
         '방문': 1170,
         '목적': 400,
         '질문': 950,
         '묵묵': 9,
         '연합뉴스': 2032,
         '김진': 16,
         '방': 636,
         '특파원': 369,
         '북한': 2884,
         '북미': 946,
         '정상회담': 626,
         '무산': 153,
         '가능성': 2021,
         '거론': 333,
         '태도': 273,
         '가운데': 1738,
         '동남아시아': 86,
         '외교': 630,
         '외무성': 103,
         '중국': 4302,
         '서우': 14,
         '首都': 2,
         '공항': 242,
         '모습': 2357,
         '날': 4483,
         '오전': 1555,
         '평양': 366,
         '발': 573,
         '고려항공': 5,
         'JS': 20,
         '편': 458,
         '이용': 1380,
         '최종': 748,
         '목적지': 61,
   

In [22]:
print(len(vocab))

53867


In [23]:
# i+2 을 한 이유는 <unk>,<number>  이미 2개 들어가 있기 때문
for i,k in enumerate(vocab):
    word2idx[k]=i+2
    idx2word[i+1]=k

다음에 할 일

train_data, train_label, test_data, test_label 데이터 만들기
- 숫자면 다 숫자 토큰 하나로 변경하기(vocab에 숫자 토큰 추가)
- 단어장에 없던 단어가 나오면 <unk> 하나로 합치기
- 기사를 2차원 벡터로 만들 것인가, 1차원 긴 벡터로 만들 것인가?
    
모델 관련    
- word embedding 적용
- 모델 세우기(Attention Is All you need 논문 참고)
- Attention vector 어떻게 줄 것인가?(1차원? 2차원?)

시각화
- position 별로 밝은 빛?
- 기사 2차원으로 보이고 밝은 빛 겹치게 할 수 있을까?
- 기사 2차원이라면 문장을 행으로 단어를 열로 해야 될 것, 각 기사마다 길이가 제각각이라 처리 필요 어떻게 처리?

In [ ]:
x_train=[]
t_train=[]
x_test=[]
t_test=[]

# 폴더 개수 8개
for i in range(NUM_CATEGORY):
    path = TEXT_DATA_DIR+'/'+str(i)
    print("%s folder 작업중..."%i)
    count = 0
    for filename in sorted(os.listdir(path)):
        filename=path+'/'+filename
        temp=Load_doc(filename)
        temp=Clean_doc(temp,vocab)
        temp=Word_To_Index(temp)
        # one hot encoding
        tempT=np.zeros(8)
        tempT[i]=1
        if count<160:
            x_train.append(temp)
            t_train.append(tempT)
        else :
            x_test.append(temp)
            t_test.append(tempT)
        count += 1
x_train=np.array(x_train)
x_test=np.array(x_test)
t_train=np.array(t_train)
t_test=np.array(t_test)
print(len(x_train))
print(len(t_train))
print(len(x_test))
print(len(t_test))

In [ ]:
# building Hierachical Attention Network
class Attention(nn.Module):
    def __init__(self,attention_size):
        super(Attention, self).__init__()
        self.attention = new_parameter(attention_size,1)
        
    def forward(self, x_in):
        # after this, we have (batch, dim1) with a diff weight per each cell
        attention_score = torch.matmul(x_in, self.attention).squeeze()
        attention_score = F.softmax(attention_score).view(x_in.size(0),x_in.size(1),1)